In [ ]:
from pdf2image import convert_from_path
from tqdm import tqdm 
from PIL import Image
import re
import os
import fitz
import pytesseract

In [ ]:
pdf_path = r'..\Downloads\Docs_C\Pdf_ListaPresentes_08.pdf'

In [ ]:
################## EXTRAIR IMAGEM DOS PDF's #######################


def exportar_paginas_para_imagens(arquivo_pdf, pasta_saida, dpi=400):
    """
    Renderiza cada página de um PDF como uma imagem rasterizada.

    Args:
        arquivo_pdf (str): Caminho para o arquivo PDF.
        pasta_saida (str): Caminho para salvar as imagens rasterizadas.
        dpi (int): Resolução em DPI (dots per inch) para a imagem.
    """
    if not os.path.exists(pasta_saida):
        os.makedirs(pasta_saida)

    try:
        pdf_documento = fitz.open(arquivo_pdf)
    except Exception as e:
        print(f"Erro ao abrir o PDF: {e}")
        return

    print(f"Exportando páginas do PDF como imagens para a pasta: {pasta_saida}")
    for numero_pagina in range(len(pdf_documento)):
        pagina = pdf_documento[numero_pagina]
        mat = fitz.Matrix(dpi / 72, dpi / 72)  # Converte DPI para a escala
        imagem_pix = pagina.get_pixmap(matrix=mat)
        nome_arquivo = os.path.join(pasta_saida, f"pagina_{numero_pagina + 1}.png")
        imagem_pix.save(nome_arquivo)
        print(f"Página {numero_pagina + 1} salva como: {nome_arquivo}")

    pdf_documento.close()
    print("Exportação concluída!")

In [ ]:
def preprocess_image(image_path):
    """
    Pré-processa uma imagem antes de aplicar OCR.

    Args:
        image_path (str): Caminho para a imagem.

    Returns:
        PIL.Image: Imagem pré-processada.
    """
    image = Image.open(image_path)
    return image

def extract_text_from_images(folder_path):
    """
    Extrai texto das imagens em uma pasta usando OCR.

    Args:
        folder_path (str): Caminho para a pasta contendo as imagens.

    Returns:
        dict: Dicionário contendo o texto extraído de cada imagem.
    """
    text_dict = {}
    # Ordenar arquivos de imagem numericamente
    image_files = sorted(
        [f for f in os.listdir(folder_path) if f.endswith('.png')],
        key=lambda x: int(re.search(r'\d+', x).group())
    )

    print("Iniciando extração de texto...")
    for i, image_file in enumerate(tqdm(image_files, desc="Extraindo texto")):  # Barra de progresso
        image_path = os.path.join(folder_path, image_file)
        preprocessed_image = preprocess_image(image_path)
        preprocessed_image = preprocessed_image.rotate(270, expand=True)
        
        # Usar OCR para extrair texto da imagem
        page_text = pytesseract.image_to_string(preprocessed_image)

        # Dividir o texto da página em linhas
        linhas = page_text.split("\n")

        # Remover linhas em branco
        linhas_filtradas = [linha.strip() for linha in linhas if linha.strip()]

        # Adicionar as linhas ao dicionário com o nome da página como chave
        text_dict[f'pagina_{i + 1}'] = linhas_filtradas

    return text_dict

def exibir_texto_estruturado(texto_extraido):
    """
    Exibe o texto extraído no formato estruturado fornecido.

    Args:
        texto_extraido (dict): Dicionário com o texto extraído de cada página.
    """
    print("\nTexto estruturado no formato desejado:\n")
    print(texto_extraido)



# Extrair texto das imagens
texto_extraido = extract_text_from_images(pasta_imagens)

# Filtrar linhas indesejadas
for pagina, linhas in texto_extraido.items():
    texto_extraido[pagina] = [
        item for item in linhas
        if item != 'Lista de Presenca' and 'Pagina' not in item
    ]

# Exibir texto no formato desejado
exibir_texto_estruturado(texto_extraido)

In [ ]:
####################### EXTRAIR NOMES ###############################


def extrair_nomes_com_cpfs(dados, chave_inicio, chave_fim):
    """
    Extrai nomes e CPFs entre duas chaves específicas em uma lista de texto,
    garantindo que nomes e CPFs estejam completos e na mesma linha.

    Args:
        dados (dict): Dicionário contendo o texto dividido por páginas.
        chave_inicio (str): Chave inicial que marca o início do intervalo.
        chave_fim (str): Chave final que marca o fim do intervalo.

    Returns:
        list: Lista de nomes com CPFs extraídos.
    """
    nomes_cpfs = []
    cpf_pattern = re.compile(r'\d{3}\.\d{3}\.\d{3}-\d{2}')
    cpf_fragment_pattern = re.compile(r'^\d{2}$|^\d{3}-\d{2}$')

    for pagina, linhas in dados.items():
        if chave_inicio in linhas and chave_fim in linhas:
            # Localizar os índices das chaves na lista
            inicio = linhas.index(chave_inicio) + 1
            fim = linhas.index(chave_fim)

            # Processar os elementos entre as chaves
            intervalo = linhas[inicio:fim]

            # Unir nomes e CPFs
            i = 0
            while i < len(intervalo):
                linha_atual = intervalo[i].strip()

                # Verificar se é uma linha de CPF
                if cpf_pattern.match(linha_atual):
                    # Adicionar CPF à última entrada de nome
                    if nomes_cpfs:
                        nomes_cpfs[-1] += f" {linha_atual}"
                elif cpf_fragment_pattern.match(linha_atual):
                    # Detectar e juntar fragmentos de CPF à última entrada
                    if nomes_cpfs and cpf_pattern.search(nomes_cpfs[-1]):
                        nomes_cpfs[-1] += linha_atual
                else:
                    # Adicionar a linha como um nome
                    nomes_cpfs.append(linha_atual)
                i += 1

    return nomes_cpfs

nomes_cpfs = extrair_nomes_com_cpfs(texto_extraido, 'Credor', 'Classe')

In [ ]:
############################### EXTRAIR CLASSES  ####################################

def extrair_nomes_entre_chaves(dados, chave_inicio, chave_fim):
    """
    Extrai os elementos entre duas chaves específicas em uma lista de texto.

    Args:
        dados (dict): Dicionário contendo o texto dividido por páginas.
        chave_inicio (str): Chave inicial que marca o início do intervalo.
        chave_fim (str): Chave final que marca o fim do intervalo.

    Returns:
        list: Lista de nomes extraídos.
    """
    nomes_extraidos = []

    for pagina, linhas in dados.items():
        if chave_inicio in linhas and chave_fim in linhas:
            # Localizar os índices das chaves na lista
            inicio = linhas.index(chave_inicio) + 1
            fim = linhas.index(chave_fim)

            # Extrair o intervalo entre as chaves
            nomes_extraidos.extend(linhas[inicio:fim])

    return nomes_extraidos




classes = extrair_nomes_entre_chaves(texto_extraido, 'Classe', 'Valor')

In [ ]:
# EXTRAIR VALOR

def extrair_valores_completos(dados, chave_inicio, chave_fim):
    """
    Extrai valores completos entre duas chaves específicas em uma lista de texto.

    Args:
        dados (dict): Dicionário contendo o texto dividido por páginas.
        chave_inicio (str): Chave inicial que marca o início do intervalo.
        chave_fim (str): Chave final que marca o fim do intervalo.

    Returns:
        list: Lista de valores extraídos.
    """
    valores_extraidos = []

    for pagina, linhas in dados.items():
        if chave_inicio in linhas and chave_fim in linhas:
            # Localizar os índices das chaves na lista
            inicio = linhas.index(chave_inicio) + 1
            fim = linhas.index(chave_fim)

            # Extrair o intervalo entre as chaves
            intervalo = linhas[inicio:fim]

            # Reconstruir valores quebrados
            valor_completo = ""
            for linha in intervalo:
                linha = linha.strip()  # Remove espaços extras
                if linha.startswith("R$"):
                    # Se já houver um valor acumulado, adiciona à lista
                    if valor_completo:
                        valores_extraidos.append(valor_completo)
                    valor_completo = linha  # Inicia novo valor
                elif valor_completo:
                    # Continua acumulando o valor se já começou com "R$"
                    valor_completo += linha
                else:
                    # Ignorar linhas que não pertencem a valores
                    continue

            # Adiciona o último valor acumulado (se existir)
            if valor_completo:
                valores_extraidos.append(valor_completo)

    return valores_extraidos


valores = extrair_valores_completos(texto_extraido, 'Valor', 'Representante')

In [ ]:
###################################     EXTRAIR  REPRESENTANTES     ####################################

def extrair_nomes_entre_chaves(dados, chave_inicio, chave_fim):
    """
    Extrai os elementos entre duas chaves específicas em uma lista de texto.

    Args:
        dados (dict): Dicionário contendo o texto dividido por páginas.
        chave_inicio (str): Chave inicial que marca o início do intervalo.
        chave_fim (str): Chave final que marca o fim do intervalo.

    Returns:
        list: Lista de nomes extraídos.
    """
    nomes_extraidos = []

    for pagina, linhas in dados.items():
        if chave_inicio in linhas and chave_fim in linhas:
            # Localizar os índices das chaves na lista
            inicio = linhas.index(chave_inicio) + 1
            fim = linhas.index(chave_fim)

            # Extrair o intervalo entre as chaves
            nomes_extraidos.extend(linhas[inicio:fim])

    return nomes_extraidos




representantes = extrair_nomes_entre_chaves(texto_extraido, 'Representante', 'CPF')

In [ ]:
def combinar_nomes_fragmentados(nomes):
    """
    Combina nomes fragmentados em uma lista, concatenando com o nome anterior.

    Args:
        nomes (list): Lista de nomes extraídos, possivelmente fragmentados.

    Returns:
        list: Lista de nomes combinados corretamente.
    """
    nomes_corrigidos = []

    for nome in nomes:
        # Se o nome parece incompleto, concatenar ao último nome na lista corrigida
        if len(nome.split()) <= 1:  # Heurística: nomes incompletos têm até 1 palavra
            if nomes_corrigidos:
                nomes_corrigidos[-1] = f"{nomes_corrigidos[-1]} {nome}".strip()
        else:
            nomes_corrigidos.append(nome)

    return nomes_corrigidos

# Combinar os nomes fragmentados
representantes_corrigidos = combinar_nomes_fragmentados(representantes)

# Resultado
print("Nomes corrigidos:")
for valor in representantes_corrigidos:
    print(valor)
print(f"Total de nomes corrigidos: {len(representantes_corrigidos)}")
print(type(representantes_corrigidos))

In [ ]:
########################## EXTRAIR CPF DO REPRESENTANTE   ##############################################


def extrair_nomes_entre_chaves(dados, chave_inicio, chave_fim):
    """
    Extrai os elementos entre duas chaves específicas em uma lista de texto.

    Args:
        dados (dict): Dicionário contendo o texto dividido por páginas.
        chave_inicio (str): Chave inicial que marca o início do intervalo.
        chave_fim (str): Chave final que marca o fim do intervalo.

    Returns:
        list: Lista de nomes extraídos.
    """
    nomes_extraidos = []

    for pagina, linhas in dados.items():
        if chave_inicio in linhas and chave_fim in linhas:
            # Localizar os índices das chaves na lista
            inicio = linhas.index(chave_inicio) + 1
            fim = linhas.index(chave_fim)

            # Extrair o intervalo entre as chaves
            nomes_extraidos.extend(linhas[inicio:fim])

    return nomes_extraidos




cpfs_representante = extrair_nomes_entre_chaves(texto_extraido, 'CPF', 'OAB')

In [ ]:
######################################### EXTRAIR OAB REPRESENTANTE ######################################



def extrair_nomes_entre_chaves(dados, chave_inicio, chave_fim):
    """
    Extrai os elementos entre duas chaves específicas em uma lista de texto.

    Args:
        dados (dict): Dicionário contendo o texto dividido por páginas.
        chave_inicio (str): Chave inicial que marca o início do intervalo.
        chave_fim (str): Chave final que marca o fim do intervalo.

    Returns:
        list: Lista de nomes extraídos.
    """
    nomes_extraidos = []

    for pagina, linhas in dados.items():
        if chave_inicio in linhas and chave_fim in linhas:
            # Localizar os índices das chaves na lista
            inicio = linhas.index(chave_inicio) + 1
            fim = linhas.index(chave_fim)

            # Extrair o intervalo entre as chaves
            nomes_extraidos.extend(linhas[inicio:fim])

    return nomes_extraidos




oab_representante = extrair_nomes_entre_chaves(texto_extraido, 'OAB', 'Data')

In [ ]:
############################ EXTRAIR DATAS ###################################


# data_patter = re.compile(r'\b\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2}')
data_patter = re.compile(r'\b\d{2}/\d{2}/\d{4} .+')
# :\d{2}\b
datas = []
for page, lines in texto_extraido.items():
    for line in lines:
        match = data_patter.search(line)
        if match:
            data = match.group()
            datas.append(data)

In [ ]:
############# Criar Dataframe e Exportar Tabela ####################

import pandas as pd


dados = {
    "Nome": nomes_final, 
    "Classe": classes, 
    "Valor": valores_final, 
    "Representante": representantes_corrigidos,
    "CPF Representante": cpfs_representante, 
    "OAB": oab_representante, 
    "Data":datas
    }

tabela = pd.DataFrame(dados)

tabela.to_excel('..\\lista_preseca.xlsx')